In [1]:
#import numpy as np
import tensorflow as tf
#import tensorflow.keras as k
#from tensorflow.keras import layers
#import sys

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import csv

#util

def get_data(symbols, dates_range, update):
    import os
    cwd = os.getcwd()
    dir = cwd + "/data/"
    df = pd.DataFrame()
    symbols.append('spy')
    for symbol in symbols:
        print("symbol", symbol)
        filename = dir + symbol + '.csv'
        data = pd.read_csv(filename, index_col=["Date"], parse_dates=['Date'])
        if df.empty:
            if 'Close' in data:
                df[symbol] = data['Close']
            else:
                df[symbol] = data['Rate']
        else:
            if 'Close' in data:
                df[symbol] = data['Close']
            else:
                df[symbol] = data['Rate']
    return df[df.index.isin(dates_range)].sort_index()

In [2]:
verbose = True
'''
# a simulator for wrapping the learner into a time frame of data features
'''
class Simulator(object):

    def __init__(self, symbols,
        start_date=dt.datetime(2008,1,1),                           #CHANGE DATES TO ACTUAL RANGE!!!
        end_date= dt.datetime(2009,1,1)):

        # frame a time period as world
        self.dates_range = pd.date_range(start_date, end_date)

        # initialize cash holdings
        init_cash = 100000

        #for visualization
        self.data_out = []

        # preprocessing time series
        # stock symbol data
        stock_symbols = symbols[:]
        symbols.append('interest_rates')
        symbols.append('vix')
        # price data
        prices_all = get_data(symbols, self.dates_range, True)

        self.stock_A = stock_symbols[0]
        self.stock_B = stock_symbols[1]

        """
        #unemployment rate
        temp_unemp = {}
        unemployment = {}
        with open('unemployment.csv') as unemp_file:
            for line in csv.reader(unemp_file, delimiter=','):
                curr_date = dt.strptime(line[0], '%B-%y')
                temp_unemp[curr_date] = line[1]
        for d in prices_all.keys():
            temp_date = dt.datetime(d.year, d.month)
            if temp_date in temp_unemp:
                unemployment[d] = temp_unemp[temp_date]
        """

        # first trading day
        self.dateIdx = 0
        self.date = prices_all.index[0]
        self.start_date = start_date
        self.end_date = end_date
        self.prices = prices_all[stock_symbols]
        self.prices_SPY = prices_all['spy']
        self.prices_VIX = prices_all['vix']
        self.prices_interest_rate = prices_all['interest_rates']

        # keep track of portfolio value as a series
        self.portfolio = {'cash': init_cash, 'a_vol': [], 'a_price': [], 'b_vol': [], 'b_price': [], 'longA': 0}
        self.port_val = self.port_value_for_output()

        # hardcode enumerating of features
        """
        self.sma = SMA(self.dates_range)
        self.bbp = BBP(self.dates_range)
        self.rsi = RSI(self.dates_range)
        """

    def init_state(self, lookback=50):
        """
        return init states of the market
        """
        states = []
        for _ in range(lookback):
            states.append(self.get_state(self.date))
            self.dateIdx += 1
            self.date = self.prices.index[self.dateIdx]

        return states

    def step(self, action):
        """
        Code indirectly based on previous code written by group member Yiding
        Zhao for the Machine Learning for Trading course
        """
        """
        Take an action, and move the date forward,
        record the reward of the action and date
        action: buy, sell, hold
        return (reward, market status)
        """
        # change state accordingly

        buy_volume = 100
        abs_return_A = 0
        pct_return_A = 0
        abs_return_B = 0
        pct_return_B = 0

        if (action == 'buy'):
            if (self.portfolio['longA'] >= 0):

                if verbose: print('---BUY WITH longA greater/equal 0')
                long_cost = buy_volume * self.prices.loc[self.date, self.stock_A]

                if verbose: print('Buying ' + str(buy_volume) + ' shares of ' + self.stock_A + ' at a price of $' + str(self.prices.loc[self.date, self.stock_A]) + ' per share, for a total cost of $' + str(long_cost) + '.')
                short_cost = buy_volume * self.prices.loc[self.date, self.stock_B]

                if verbose: print('Shorting ' + str(buy_volume) + ' shares of ' + self.stock_B + ' at a price of $' + str(self.prices.loc[self.date, self.stock_B]) + ' per share, for a total cost of $' + str(short_cost) + '.')
                total_cost = short_cost + long_cost

                if verbose: print('Total cost is $' + str(total_cost))

                if verbose: print('Pre-transaction cash is $' + str(self.portfolio['cash']))
                self.portfolio['cash'] -= total_cost

                if verbose: print('Post-transaction cash is $' + str(self.portfolio['cash']))

                self.portfolio['a_vol'].append(buy_volume)
                self.portfolio['a_price'].append(self.prices.loc[self.date, self.stock_A])
                self.portfolio['b_vol'].append(buy_volume)
                self.portfolio['b_price'].append(self.prices.loc[self.date, self.stock_B])
                self.portfolio['longA'] = 1

                if verbose: print(self.portfolio)
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()

                if verbose: print(self.portfolio)
                reward = self.port_val - old_port_val
                if verbose: print('---END OF BUY WITH longA greater/equal 0')

            else: #longA < 0 --> sell in reverse
                if verbose: print('---BUYING (ACTUALLY SELLING) WITH longA < 0')
                if verbose: print('Selling our long investment of ' + str(self.portfolio['b_vol'][0]) + ' shares for $' + str(self.prices.loc[self.date, self.stock_B]))
                long_initial = self.portfolio['b_vol'][0] * self.portfolio['b_price'][0]
                long_return = self.portfolio['b_vol'].pop(0) * self.prices.loc[self.date, self.stock_B]
                abs_return_B = long_return - long_initial
                pct_return_B = float(abs_return_B) / long_initial
                if verbose:print('Long initial is ' + str(long_initial))
                if verbose:print('Long return is ' + str(long_return))
                if verbose:print('Long return - long initial = ' + str(abs_return_B))
                if verbose:print('(long return - long initial) / long initial = ' + str(pct_return_B))
                self.portfolio['b_price'].pop(0)
                if verbose: print('Return is $' + str(long_return))
                if verbose: print('Cover our long investment of ' + str(self.portfolio['a_vol'][0]) + ' shares that we bought for $' + str(self.portfolio['a_price'][0]) + ' and add to it a gain of $' + str((self.portfolio['a_price'][0] - self.prices.loc[self.date, self.stock_A])) + ' (' + str((self.portfolio['a_price'][0])) + ' - ' + str(self.prices.loc[self.date, self.stock_A]) + ') for each of our ' + str(self.portfolio['a_vol'][0]) + ' stocks.')
                short_initial = self.portfolio['a_vol'][0] * self.portfolio['a_price'][0]
                abs_return_A = (self.portfolio['a_vol'][0] * (self.portfolio['a_price'][0] - self.prices.loc[self.date, self.stock_A]))
                short_return = self.portfolio['a_vol'][0] * self.portfolio['a_price'][0]
                short_return += (self.portfolio['a_vol'].pop(0) * (self.portfolio['a_price'].pop(0) - self.prices.loc[self.date, self.stock_A]))
                pct_return_A = float(abs_return_A) / short_initial
                if verbose:print('Short initial is ' + str(short_initial))
                if verbose:print('Short return is ' + str(short_return))
                if verbose:print('Absolute return for short is ' + str(abs_return_A))
                if verbose:print('Percetn return for short is ' + str(pct_return_A))
                if verbose: print('Short return is $' + str(short_return))
                if verbose: print('Old cash is $' + str(self.portfolio['cash']))
                new_cash = self.portfolio['cash'] + long_return + short_return
                self.portfolio['cash'] = new_cash
                if verbose: print('New cash is $' + str(self.portfolio['cash']))
                self.portfolio['longA'] = -1 if (len(self.portfolio['a_vol']) > 0) else 0
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                if verbose: print(self.portfolio)
                reward = self.port_val - old_port_val
                if verbose: print('Old portfolio value is $' + str(old_port_val))
                if verbose: print('New portfolio value is $' + str(self.port_val))
                if verbose: print('Reward is $' + str(reward))

        elif (action == 'sell'):
            if (self.portfolio['longA'] > 0):
                if verbose: print('---SELLING WITH longA > 0')
                if verbose: print('Selling our long investment of ' + str(self.portfolio['a_vol'][0]) + ' shares for $' + str(self.prices.loc[self.date, self.stock_A]))
                long_initial = self.portfolio['a_vol'][0] * self.portfolio['a_price'][0]
                long_return = self.portfolio['a_vol'].pop(0) * self.prices.loc[self.date, self.stock_A]
                abs_return_A = long_return - long_initial
                pct_return_A = float(abs_return_A) / long_initial
                self.portfolio['a_price'].pop(0)
                if verbose: print('Return is $' + str(long_return))
                if verbose: print('Cover our long investment of ' + str(self.portfolio['b_vol'][0]) + ' shares that we bought for $' + str(self.portfolio['b_price'][0]) + ' and add to it a gain of $' + str((self.portfolio['b_price'][0] - self.prices.loc[self.date, self.stock_B])) + ' (' + str((self.portfolio['b_price'][0])) + ' - ' + str(self.prices.loc[self.date, self.stock_B]) + ') for each of our ' + str(self.portfolio['b_vol'][0]) + ' stocks.')
                short_initial = self.portfolio['b_vol'][0] * self.portfolio['b_price'][0]
                abs_return_B = (self.portfolio['b_vol'][0] * (self.portfolio['b_price'][0] - self.prices.loc[self.date, self.stock_B]))
                short_return = self.portfolio['b_vol'][0] * self.portfolio['b_price'][0]
                short_return += (self.portfolio['b_vol'].pop(0) * (self.portfolio['b_price'].pop(0) - self.prices.loc[self.date, self.stock_B]))
                pct_return_B = float(abs_return_B) / short_initial
                if verbose: print('Short return is $' + str(short_return))
                if verbose: print('Old cash is $' + str(self.portfolio['cash']))
                new_cash = self.portfolio['cash'] + long_return + short_return
                self.portfolio['cash'] = new_cash
                if verbose: print('New cash is $' + str(self.portfolio['cash']))
                self.portfolio['longA'] = 1 if (len(self.portfolio['a_vol']) > 0) else 0
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                if verbose: print(self.portfolio)
                reward = self.port_val - old_port_val
                if verbose: print('Old portfolio value is $' + str(old_port_val))
                if verbose: print('New portfolio value is $' + str(self.port_val))
                if verbose: print('Reward is $' + str(reward))
            else: # longA <= 0 --> buy in reverse
                if verbose: print('---SELLING (ACTUALLY BUYING) WITH long <= 0')
                long_cost = buy_volume * self.prices.loc[self.date, self.stock_B]
                if verbose: print('Buying ' + str(buy_volume) + ' shares of ' + self.stock_B + ' at a price of $' + str(self.prices.loc[self.date, self.stock_B]) + ' per share, for a total cost of $' + str(long_cost) + '.')
                short_cost = buy_volume * self.prices.loc[self.date, self.stock_A]
                if verbose: print('Shorting ' + str(buy_volume) + ' shares of ' + self.stock_A + ' at a price of $' + str(self.prices.loc[self.date, self.stock_A]) + ' per share, for a total cost of $' + str(short_cost) + '.')
                total_cost = short_cost + long_cost
                if verbose: print('Total cost is $' + str(total_cost))
                if verbose: print('Pre-transaction cash is $' + str(self.portfolio['cash']))
                self.portfolio['cash'] -= total_cost
                if verbose: print('Post-transaction cash is $' + str(self.portfolio['cash']))
                self.portfolio['a_vol'].append(buy_volume)
                self.portfolio['a_price'].append(self.prices.loc[self.date, self.stock_A])
                self.portfolio['b_vol'].append(buy_volume)
                self.portfolio['b_price'].append(self.prices.loc[self.date, self.stock_B])
                self.portfolio['longA'] = -1
                if verbose: print(self.portfolio)
                old_port_val = self.port_val
                self.port_val = self.port_value_for_output()
                if verbose: print(self.portfolio)
                reward = self.port_val - old_port_val
                if verbose: print('---END OF SELLING (ACTUALLY BUYING) WITH long <= 0')
        else: #hold
            if verbose: print('---HOLDING')
            old_port_val = self.port_val
            self.port_val = self.port_value_for_output()
            if verbose: print(self.portfolio)
            reward = self.port_val - old_port_val
        #self.port_val = self.port_value_for_output()
        print("port value", self.port_val)
        self.data_out.append(self.date.isoformat()[0:10] + ',' + str(self.prices.loc[self.date, self.stock_A]) + ',' + str(self.prices.loc[self.date, self.stock_B]) + ',' + action + ',' + str(abs_return_A) + ',' +  str(pct_return_A) + ',' + str(abs_return_B) + ',' + str(pct_return_B) + ',' + str((self.prices.loc[self.date, self.stock_A] - self.prices.loc[self.date, self.stock_B])) + ',' + str(self.prices_interest_rate[self.date]) + ',' + str(self.prices_SPY[self.date]) + ',' + str(self.prices_VIX[self.date]) + ',' + str(self.port_val))
        if verbose: print(self.data_out)
        state = self.get_state(self.date)
        if verbose: print(state)
        self.dateIdx += 1
        if self.dateIdx < len(self.prices.index):
            self.date = self.prices.index[self.dateIdx]
        if verbose: print(self.get_state(self.date))
        if verbose: print('New date is')
        if verbose: print(self.date)
        if verbose: print('Reward is ' + str(reward))
        return (reward, state)

    def get_state(self, date):
        """
        return state of the market, i.e. prices of certain symbols,
        number of shares hold
        """
        if date not in self.dates_range:
            if verbose: print('Date was out of bounds.')
            if verbose: print(date)
            exit

        # a vector of features
        if (date == self.prices.index[-1]):
            file_name = "data_for_vis_%s.csv" % dt.datetime.now().strftime("%H-%M-%S")
            print ("saving to", file_name)
            file = open(file_name, 'w');
            for line in self.data_out:
                file.write(line);
                file.write('\n')
            file.close()
        return [self.prices.loc[date, self.stock_A]/self.prices.loc[self.prices.index[0], self.stock_A] - self.prices.loc[date, self.stock_B]/self.prices.loc[self.prices.index[0], self.stock_B],
            self.prices_interest_rate[date]/self.prices_interest_rate[0] - 1,
            self.prices_SPY[date]/self.prices_SPY[0] - 1,
            self.prices_VIX[date]/self.prices_VIX[0] - 1,
            self.port_val / 100000.0 - 1,
            ]

    # calculate the current value of cash and stock holdings
    def port_value(self):
        value = self.portfolio['cash']
        if (len(self.portfolio['a_vol']) > 0):
            for i in range(len(self.portfolio['a_vol'])):
                value += (self.portfolio['a_vol'][i] * self.portfolio['a_price'][i])
        if (len(self.portfolio['b_vol']) > 0):
            for i in range(len(self.portfolio['b_vol'])):
                value += (self.portfolio['b_vol'][i] * self.portfolio['b_price'][i])
        return value

    # alternate calculation of the current value of cash and stock holdings
    def port_value_for_output(self):
        value = self.portfolio['cash']
        if (self.portfolio['longA'] > 0):
            value += (sum(self.portfolio['a_vol']) * self.prices.loc[self.date, self.stock_A])
            for i in range(len(self.portfolio['b_vol'])):
                value += (self.portfolio['b_vol'][i] * self.portfolio['b_price'][i])
                value += (self.portfolio['b_vol'][i] * (self.portfolio['b_price'][i] - self.prices.loc[self.date, self.stock_B]))
        if (self.portfolio['longA'] < 0):
            value += (sum(self.portfolio['b_vol']) * self.prices.loc[self.date, self.stock_B])
            for i in range(len(self.portfolio['a_vol'])):
                value += (self.portfolio['a_vol'][i] * self.portfolio['a_price'][i])
                value += (self.portfolio['a_vol'][i] * (self.portfolio['a_price'][i] - self.prices.loc[self.date, self.stock_A]))
        return value

    def has_more(self):
        if ((self.dateIdx < len(self.prices.index)) == False):
            print('\n\n\n*****')
            print(self.baseline())
            print('*****\n\n\n')
        return self.dateIdx < len(self.prices.index)

    def baseline(self):
        num_shares = 100000 / self.prices_SPY[0]
        return num_shares * self.prices_SPY[-1]

def main():
    # TO DO
    sim = Simulator(['scg', 'wec'])
    sim.init_state()
    #sim.get_state(sim.prices.index[-1])
    sim.step('buy')
    sim.step('hold')
    sim.step('buy')
    sim.step('sell')
    sim.step('sell')
    sim.step('hold')
    sim.step('hold')
    sim.step('sell')
    sim.step('sell')
    sim.step('buy')
    sim.step('buy')


if __name__ == '__main__':
    main()

symbol scg
symbol wec
symbol interest_rates
symbol vix
symbol spy
---BUY WITH longA greater/equal 0
Buying 100 shares of scg at a price of $36.77 per share, for a total cost of $3677.0000000000005.
Shorting 100 shares of wec at a price of $43.049999 per share, for a total cost of $4304.9999.
Total cost is $7981.999900000001
Pre-transaction cash is $100000
Post-transaction cash is $92018.0001
{'cash': 92018.0001, 'a_vol': [100], 'a_price': [36.77], 'b_vol': [100], 'b_price': [43.049999], 'longA': 1}
{'cash': 92018.0001, 'a_vol': [100], 'a_price': [36.77], 'b_vol': [100], 'b_price': [43.049999], 'longA': 1}
---END OF BUY WITH longA greater/equal 0
port value 100000.0
['2008-03-14,36.77,43.049999,buy,0,0,0,0,-6.279998999999997,2.99,129.610001,31.16,100000.0']
[-0.01460351659829906, -0.2725060827250608, -0.10570615290100782, 0.34484246870953816, 0.0]
[-0.02568782985131124, -0.34549878345498786, -0.11474498587742288, 0.39145455330168333, 0.0]
New date is
2008-03-17 00:00:00
Reward is 0.0
--

SyntaxError: invalid syntax (Temp/ipykernel_5148/3954194794.py, line 14)

In [49]:
import numpy as np
import sys


class PolicyGradientAgent(object):
    def __init__(self, n_batch=5, batch_size=64, update_cycle=3, update_target_cycle= 3, lookback=[]):
        #self.n_bath = 1 # number of batches to run in each update
        self.batch_size = batch_size # number of x, y pairs to fit model
        self.update_cycle = update_cycle # cyle to update actor and critics model, i.e. every 5 days
        self.update_target_cycle = update_target_cycle #cycle for update target model
        self.lookback_size = len(lookback)
        self.n_feature = len(lookback[0])
        self.states = lookback[:]
        self.choices = []
        self.rewards = []
        self.model = LSTM(self.lookback_size, len(lookback[0]))

        self.size_of_replay = self.batch_size*4

        #initialize critic network
        self.critic_agent = CriticsAgent(batch_size = self.batch_size, critic_update_cycle=self.update_cycle, target_update_cycle = self.update_target_cycle, lookback = lookback)

    def reset(self, lookback):
        # clear all rewards and actions
        self.lookback_size = len(lookback)
        self.states = lookback[:]
        self.choices = []
        self.rewards = []

    def init_query(self):
        probs = self.model.predict(self.states)
        print ("probabilities", probs)
        self.action = np.random.choice(np.arange(3), p=probs)
        return self.action

    def query(self, new_state, reward):


        # update critics's experience repaly
        self.critic_agent.update_history(self.states, self.rewards)


        self.choices.append(self.action)
        self.rewards.append(reward)
        # for i in range(len(self.rewards) - 2, -1, -1):
        #     reward *= 0.95
        #     self.rewards[i] += reward


        if len(self.choices) % self.update_cycle == 0 and len(self.choices) >= self.batch_size * 1.25:
            #update actor, choose random sample
            random_sample = np.random.choice(np.arange(max(0,len(self.rewards) - self.size_of_replay - 1), len(self.rewards) - 1), self.batch_size)
            print ("updating actor")
            self.update(random_sample)
            #update critic
            print ("updating crititc")
            self.critic_agent.update_critic(random_sample)

        if len(self.choices) % self.update_target_cycle == 0 and len(self.choices) >= self.batch_size * 1.25:
            #update target network
            print ("updating target")
            self.critic_agent.update_target()


        print ("new state", new_state)
        self.states.append(new_state)
        probs = self.model.predict(self.states[-self.lookback_size:])

        print ("probabilities", probs)
        if np.isnan(probs[0]):
            sys.exit()
        self.action = np.random.choice(3, p=probs)

        # update critics's experience repaly
        self.critic_agent.update_history(self.states, self.rewards)

        return self.action

    def update(self, random_sample):
        # for j in range(min(self.n_bath, len(self.rewards)/self.batch_size)):
        #     offset = j * self.batch_size
        #     Input = []
        #     if not offset:
        #         Input.append(self.states[- self.lookback_size -offset:])
        #     else:
        #         Input.append(self.states[- self.lookback_size - offset: -offset])
        #     for i in range(1, self.batch_size):
        #         Input.append(self.states[- i - self.lookback_size - offset: - i - offset])
        #     Input = np.array(Input).astype(theano.config.floatX)

        #     rewards = self.rewards[-self.batch_size - offset: -offset] if offset else self.rewards[-self.batch_size:]
        #     rewards = np.array(rewards).astype(theano.config.floatX)

        #     choices = self.choices[-self.batch_size - offset: -offset] if offset else self.choices[-self.batch_size:]
        #     choices = np.array(choices).astype(np.int32)
        #     self.model.train(Input, choices, rewards)


        Input = np.zeros((self.batch_size, self.lookback_size, self.n_feature), dtype = theano.config.floatX)
        Input_next = np.zeros((self.batch_size, self.lookback_size, self.n_feature), dtype = theano.config.floatX)
        choices = np.array(self.choices, dtype = np.int32)[random_sample]
        # td_advantage = np.zeros(self.batch_size, dtype = np.int32)

        for i in range(self.batch_size):

            random_id = random_sample[i]
            state_cur = np.array(self.states[random_id: random_id+self.lookback_size], dtype = theano.config.floatX)
            state_next = np.array(self.states[random_id+1: random_id+self.lookback_size+1], dtype = theano.config.floatX)

            Input[i,:,:] = state_cur
            Input_next[i,:,:] = state_next
            #td error estimating the advantage function, to multiply policy gradient
        td_advantage = np.array(self.rewards)[random_sample] + 0.95*self.critic_agent.target_batch_query(Input_next) - self.critic_agent.batch_query(Input)
        self.model.train(Input, choices, td_advantage)


#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import LSTM as lstm
#from tensorflow.keras import Model, Sequential
#from keras import backend as K
#from tensorflow.keras.optimizers import Adagrad, Adam
from keras.layers import Dense, Input
from keras.layers import LSTM as lstm
from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import Adagrad, Adam

class model(object):
    def __init__(self, seq_len, n_feature):

        self.seq_len = seq_len
        self.n_feature = n_feature
        self.model = self.buildnetwork()


    def buildnetwork(self):
        model = Sequential()
        model.add(lstm(20, dropout=0.2,input_shape = (self.seq_len, self.n_feature)))
        model.add(Dense(1, activation=None))
        model.compile(loss='mean_squared_error', optimizer=Adagrad(lr=0.002,clipvalue=10), metrics=['mean_squared_error'])

        return model

    def predict(self, x):
        # x is a tensor having shape (1, seq_len, n_feature)
        return self.model.predict(x)

    def get_weights(self):
        return self.model.get_weights()

    def train(self, x_batch, target_batch):
        # x_batch is a tensor having shape (batch_size, seq_len, n_feature)
        # target batch is a tensor having shape (batch_size, 1)
        self.model.train_on_batch(x_batch, target_batch)

class target_model(model):

    def set_weights(self, weights):
        #weights should be returned by "get_weights" method in Critic network
        self.model.set_weights(weights)


class CriticsAgent(object):
    def __init__(self, batch_size=100, critic_update_cycle=2, target_update_cycle=5,lookback=[]):
        self.batch_size = batch_size

        self.states = lookback[:]
        self.rewards = []

        self.n_feature = len(lookback[0])
        self.lookback_size = len(lookback)

        self.critic_model = model(self.lookback_size, len(lookback[0]))
        self.target_model = target_model(self.lookback_size, len(lookback[0]))

        self.critic_update_cycle = critic_update_cycle
        self.target_update_cycle = target_update_cycle

        self.gamma = 0.95

        self.count = 0

        self.init_lookback = lookback

    def update_critic(self, random_sample):
        #random_sample = np.random.choice(np.arange(len(self.rewards)-1), self.batch_size)

        states_batch = np.zeros((self.batch_size, self.lookback_size, self.n_feature), dtype = "float32")
        states_next_batch = np.zeros((self.batch_size, self.lookback_size, self.n_feature),dtype = "float32")

        #print random_sample

        for i in range(self.batch_size):
            random_id = random_sample[i]
            states_batch[i,:,:] =np.array(self.states[random_id:random_id+self.lookback_size]).astype("float32")
            states_next_batch[i,:,:] =np.array(self.states[random_id + 1:(random_id+self.lookback_size +1)]).astype("float32")

        reward_batch = np.array([self.rewards[i] for i in random_sample]).astype("float32")
        #using target model to predict
        target_value = self.target_model.predict(states_next_batch).flatten()*self.gamma + reward_batch

        self.critic_model.train(states_batch, target_value.reshape(self.batch_size,1))


    def query(self, state):
        #query state for one day (shape: (lookback_size, n_feature))
        value = self.critic_model.predict(state.reshape(1,self.lookback_size, self.n_feature))[0][0]
        #print "predicted_value:", value
        return value

    def batch_query(self, states):
        values = self.critic_model.predict(states.reshape(self.batch_size, self.lookback_size, self.n_feature))[0]
        return values


    def target_batch_query(self, states):
        values = self.target_model.predict(states.reshape(self.batch_size, self.lookback_size, self.n_feature))[0]
        return values

    def update_target(self):
        weights = self.critic_model.get_weights()
        self.target_model.set_weights(weights)


    def update_history(self, states, rewards):
        self.states = states
        self.rewards = rewards

    # def test(self):
    #     data = np.array(self.init_lookback).reshape(1,200,5).astype("float32")
    #     print self.critic_model.predict(data)
    #     self.update_target()
    #     print self.target_model.predict(data)



In [80]:
import tensorflow as tf
class LSTM(object):
    def __init__(self, seq_len, n_feature, n_actions = 3):
        self.Inputs = Input(shape=(seq_len, n_feature), dtype=tf.float32)
        self.buildNetwork()
        
        #self.q = tf.placeholder(shape=None, dtype=tf.float32) 
        self.q = Input(shape=(1,), dtype=tf.float32)
        #self.actions = tf.placeholder(shape=(None, n_actions), dtype=tf.float32)
        
        #self.actions_prob = K.softmax(self.last_layer)
        self.actions_prob = self.last_layer
        self.output_fn = Model(inputs = self.Inputs, outputs = self.last_layer)
        self.output_fn.summary()
        
        choices = tf.placeholder(shape=(1,), dtype=tf.int32)
        
        px = self.actions_prob[K.arange(self.actions_prob.shape[1]), choices]
        print(px, px.shape)
        ###self.log_policy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.last_layer, labels=self.actions)
        log_px = tf.math.log(px)
        print(log_px, log_px.shape)
        print(self.q, self.q.shape)
        #sys.exit()
        ###self.cost = tf.reduce_mean(self.q * self.log_policy) 
        cost = layers.dot([-self.q, log_px], axes = 1)
        print(cost)
        
        self.train_fn = Model(inputs = [self.Inputs, choices, self.q], outputs = [cost, px, log_px])
        self.optimizer = tf.train.AdagradOptimizer(learning_rate=0.0008).minimize(cost)
        #self.train_fn.compile(optimizer = Adagrad(learning_rate = 0.0008), loss = cost, metrics = 'accuracy')
    
    
    def buildNetwork(self):
        inputs = self.Inputs
        x = lstm(20, dropout= 0.2)(self.Inputs)
        self.last_layer = Dense(3, activation = 'softmax')(x)
        
        return self.last_layer
    
    def predict(self, x):
        x = np.array([x]).astype('float32')
        prediction = self.output_fn(x)
        return prediction[0]

    def train(self, Input, choices, rewards):
        if rewards.std() != 0:
            rewards = (rewards - rewards.mean()) / rewards.std()
        cost, px, log_px = self.train_fn(Input, choices, rewards)
        if np.isnan(cost) or any(np.isnan(log_px)):
            print (rewards, "\n", cost, px, log_px)
        

In [47]:
class LSTM(object):
    def __init__(self, seq_len, n_feature, n_actions = 3):
        self.Inputs = k.Input(shape=(seq_len, n_feature), dtype=tf.float32)
        self.buildNetwork()
        
        self.q = tf.placeholder(shape=(1,), dtype=tf.float32, name='pg_q') 
        self.actions = tf.placeholder(shape=(None, n_actions), dtype=tf.float32, name='pg_actions')
        choices = tf.placeholder(shape=None, dtype=tf.int32)
                
        ###_layer = self.states
        #for l in hidden_layers_size:
        #    _layer = tf.layers.dense(inputs=_layer, units=l, activation=tf.nn.relu,
        #                             kernel_initializer=tf.contrib.layers.xavier_initializer())
        #self.last_layer = tf.layers.dense(inputs=_layer, units=num_of_actions, activation=None, # Linear activation
        ###                                  kernel_initializer=tf.contrib.layers.xavier_initializer())
        self.action_prob = tf.nn.softmax(self.last_layer)
        self.log_policy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.last_layer, labels=self.actions)
        self.cost = tf.reduce_mean(self.q * self.log_policy) 
        self.optimizer = tf.train.AdagradOptimizer(learning_rate=0.0008).minimize(self.cost)
        px = self.action_prob
        log_px = self.log_policy
        cost = self.cost
        
        
        #self.train_fn = Model.compile(optimizer = Adagrad(learning_rate = 0.0008), loss = self.cost, metrics = 'accuracy')
        self.train_fn = Model(inputs = [self.Inputs, choices, self.q], outputs = [cost, px, log_px])
        
    
    
    def buildNetwork(self):
        x = lstm(20, dropout= 0.2, input_shape = (self.seq_len, self.n_feature))(self.Inputs)
        self.last_layer = Dense(3, activation = None)(x)
        
        return self.last_layer
    
    def predict(self, x):
        x = np.array([x]).astype('float32')
        prediction = self.output_fn(x)
        return prediction[0]

    def train(self, Input, choices, rewards):
        if rewards.std() != 0:
            rewards = (rewards - rewards.mean()) / rewards.std()
        cost, px, log_px = self.train_fn(Input, choices, rewards)
        if np.isnan(cost) or any(np.isnan(log_px)):
            print (rewards, "\n", cost, px, log_px)
        

In [24]:
from keras import layers

In [63]:
#keras
class LSTM(object):
    def __init__(self, seq_len, n_feature, n_actions = 3):
        self.Inputs = Input(shape=(seq_len, n_feature), dtype='float32')
        self.buildNetwork()
        
        #self.q = tf.placeholder(shape=None, dtype=tf.float32) 
        self.q = Input(shape=(1,), dtype='float32')
        #self.actions = tf.placeholder(shape=(None, n_actions), dtype=tf.float32)
        
        #self.actions_prob = tf.nn.softmax(self.last_layer)
        self.actions_prob = self.last_layer
        self.output_fn = Model(inputs = self.Inputs, outputs = self.actions_prob)
        self.output_fn.summary()
        
        choices = Input(shape=(1,), dtype='int32')
        
        px = self.actions_prob[np.arange(self.actions_prob.shape[0].value), choices]
        print(px, px.shape)
        ###self.log_policy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.last_layer, labels=self.actions)
        log_px = tf.math.log(px)
        print(log_px, log_px.shape)
        print(self.q, self.q.shape)
        #sys.exit()
        ###self.cost = tf.reduce_mean(self.q * self.log_policy) 
        cost = K.dot(-self.q, log_px)
        print(cost)
        self.train_fn = Model(inputs = [self.Inputs, choices, self.q], outputs = [cost, px, log_px])
        self.optimizer = tf.train.AdagradOptimizer(learning_rate=0.0008).minimize(cost)
        #self.train_fn.compile(optimizer = Adagrad(learning_rate = 0.0008), loss = cost, metrics = 'accuracy')
    
    
    def buildNetwork(self):
        inputs = self.Inputs
        x = lstm(20, dropout= 0.2)(self.Inputs)
        self.last_layer = Dense(3, activation = 'softmax')(x)
        
        return self.last_layer
    
    def predict(self, x):
        x = np.array([x]).astype('float32')
        prediction = self.output_fn(x)
        return prediction[0]

    def train(self, Input, choices, rewards):
        if rewards.std() != 0:
            rewards = (rewards - rewards.mean()) / rewards.std()
        cost, px, log_px = self.train_fn(Input, choices, rewards)
        if np.isnan(cost) or any(np.isnan(log_px)):
            print (rewards, "\n", cost, px, log_px)
        

In [81]:
import datetime as dt
import numpy as np
def main():
    actions = ["buy", "sell", "hold"]

    n_iter = 5
    for i in range(n_iter):

        env_train = Simulator(['scg', 'wec'], dt.datetime(2002, 1, 4), dt.datetime(2016, 12, 31))

        agent = PolicyGradientAgent(lookback=env_train.init_state())
        #critic_agent = CriticsAgent(lookback=env.init_state())
        action = agent.init_query()


        while env_train.has_more():
        	action = actions[action] # map action from id to name
        	print ("Runner: Taking action", env_train.date, action)
        	reward, state = env_train.step(action)
        	action = agent.query(state, reward)
'''
    env_test = Simulator(['scg', 'wec'], dt.datetime(2013, 12, 30), dt.datetime(2016, 11, 30))
    agent.reset(lookback=env_test.init_state())
    while env_test.has_more():
        action = actions[action] # map action from id to name
        print "Runner: Taking action", env_test.date, action
        reward, state = env_test.step(action)
        action = agent.query(state, reward)
'''
if __name__ == '__main__':
    main()

symbol scg
symbol wec
symbol interest_rates
symbol vix
symbol spy
Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_65 (InputLayer)        (None, 50, 5)             0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 20)                2080      
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 63        
Total params: 2,143
Trainable params: 2,143
Non-trainable params: 0
_________________________________________________________________


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got <tf.Tensor 'arange_9:0' shape=(3,) dtype=int32>

In [ ]:
from stable_baselines import A2C
from stable_baselines.common.policies import mlplstmpolicy

ERROR! Session/line number was not unique in database. History logging moved to new session 49


D:\Anaconda\envs\pairstr\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
